<a href="https://colab.research.google.com/github/rautaditya2606/Santander-Customer-Transaction-Prediction/blob/main/santander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import matplotlib
%matplotlib inline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import opendatasets as od
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor, XGBClassifier
import joblib
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier



sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10,6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
database_url = 'https://www.kaggle.com/competitions/santander-customer-transaction-prediction/overview'
od.download(database_url)

Skipping, found downloaded files in "./santander-customer-transaction-prediction" (use force=True to force download)


In [ ]:
data_dir = '/content/santander-customer-transaction-prediction'

In [ ]:
raw_df = pd.read_csv('/content/santander-customer-transaction-prediction/train.csv')
test_df = pd.read_csv('/content/santander-customer-transaction-prediction/test.csv')
submission_df = pd.read_csv('/content/santander-customer-transaction-prediction/sample_submission.csv')

In [ ]:
raw_df

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,6.1415,13.2305,3.9901,0.9388,18.0249,-1.7939,2.1661,8.5326,16.6660,-17.8661
199996,train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,4.9611,4.6549,0.6998,1.8341,22.2717,1.7337,-2.1651,6.7419,15.9054,0.3388
199997,train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,4.0651,5.4414,3.1032,4.8793,23.5311,-1.5736,1.2832,8.7155,13.8329,4.1995
199998,train_199998,0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,2.6840,8.6587,2.7337,11.1178,20.4158,-0.0786,6.7980,10.0342,15.5289,-13.9001


In [ ]:
# raw_df.corr(numeric_only=True)['target'].sort_values(ascending=False)

In [ ]:
raw_df.select_dtypes(include=['float', 'int']).columns

Index(['target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6',
       'var_7', 'var_8',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=201)

In [ ]:
inputs_cols = raw_df.columns.drop(['ID_code', 'target'])
target_col = 'target'

In [ ]:
inputs = raw_df[inputs_cols]
target = raw_df[target_col]

In [ ]:
train_inputs, val_inputs, train_target, val_target = train_test_split(inputs, target, test_size=0.3, random_state=42, stratify=target)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
train_inputs = scaler.fit_transform(train_inputs)

In [ ]:
val_inputs = scaler.fit_transform(val_inputs)

In [ ]:
# param_dict = {
#     'max_depth' : [3,4,5,6,7,8,10],
#     'learning_rate' : [0.05, 0.1, 0.15, 0.2],
#     'n_estimators' : [300, 500, 1000],
#     'subsample': [0.6, 0.8, 1.0],
#     'comsample_bytree': [0.6, 0.8, 1.0],
#     'min_child_weight': [1,3,5,7,9],
#     'gamma': [0, 0.1, 0.5, 1, 2],
#     'reg_alpha': [0, 0.1, 1, 2],
#     'reg_lambda': [0, 0.1, 1, 2]
# }

# search = RandomizedSearchCV(
#     XGBClassifier(
#         n_job=-1,
#         eval_metric= 'auc',
#         use_label_encoder = False,
#     ),
#     param_distributions = param_dict,
#     n_iter = 30,
#     scoring='roc_auc',
#     cv=3,
#     verbose = 2,
#     random_state=42
# )

# search.fit(train_inputs, train_target)
# print('Best Param: ', search.best_params_)

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(
    n_jobs=-1,
    n_estimators=2000,
    max_depth=3,
    learning_rate=0.1,
    subsample=1.0,
    colsample_bytree=0.8,
    min_child_samples=9,
    min_split_gain=0.5,
    reg_alpha=1,
    reg_lambda=2,
    random_state=42,
    verbose=-1
)

model.fit(train_inputs, train_target, eval_set=[(val_inputs, val_target)], eval_metric='auc')


KeyboardInterrupt: 

In [ ]:
# train_pred = model.predict(train_inputs)

In [ ]:
train_prob = model.predict_proba(train_inputs)[:,-1]
val_prob = model.predict_proba(val_inputs)[:,-1]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
print('Train AUC', roc_auc_score(train_target, train_prob))
print('Val AUC', roc_auc_score(val_target, val_prob))#Val AUC 0.8887801404585796

Train AUC 0.9706901547279417
Val AUC 0.8887735053674278


In [ ]:
!pip install -U xgboost
